# RLWE Homomorphic Encryption Example
This Jupyter Notebook provides an example of using the RLWE homomorphic encryption scheme for encrypting and decrypting binary vectors.

## Import Libraries
We begin by importing the necessary libraries for our example.

In [1]:
import rlwe_he_scheme_updated as rlwe_updated
import numpy as np

## Scheme Parameters

We define the parameters for the RLWE encryption scheme.

$$
\begin{align*}
\text{Polynomial Modulus Degree (n)} &: n = 2^3 = 8 \\
\text{Ciphertext Modulus (q)} &: q = 2^{14} = 16384 \\
\text{Plaintext Modulus (t)} &: t = 2 \\
\text{Base for relin\_v1 (T)} &: T = \sqrt{q} = \sqrt{16384} = 128 \\
\text{Modulusswitching Modulus (p)} &: p = q^3 = 4398046511104 \\
\text{Polynomial Modulus (poly\_mod)} &: \text{poly\_mod} = [1, 0, 0, \ldots, 0, 1] \\
\text{Standard Deviation (std1)} &: \text{std1} = 1 \\
\text{Standard Deviation (std2)} &: \text{std2} = 1
\end{align*}
$$

These parameters are essential for the RLWE homomorphic encryption scheme and are used for various encryption and evaluation operations.


In [2]:
# Scheme's parameters
# polynomial modulus degree
n = 2 ** 3
# ciphertext modulus
q = 2 ** 14
# plaintext modulus
t = 2
# base for relin_v1
T = int(np.sqrt(q))
# modulusswitching modulus
p = q ** 3

# polynomial modulus
poly_mod = np.array([1] + [0] * (n - 1) + [1])

# standard deviation for the error in the encryption
std1 = 1
# standard deviation for the error in the evaluateKeyGen_v2
std2 = 1
n, q, t, T, p, poly_mod, std1, std2

(8, 16384, 2, 128, 4398046511104, array([1, 0, 0, 0, 0, 0, 0, 0, 1]), 1, 1)

## Key Generation
The key generation process involves generating a public key $ pk $ and a secret key $ sk $. The public key is used for encryption, and the secret key is used for decryption.

### Generating Secret and Public Keys

The secret key $ sk $ is a binary polynomial. The public key $ pk $ is generated as follows:

$$
\begin{align*}
sk &: \text{Binary secret key polynomial} \\
a &: \text{Uniformly random polynomial} \\
e &: \text{Error polynomial sampled from a normal distribution} \\
b &: a \cdot sk - e \mod q
\end{align*}
$$

After the key generation process, we obtain the secret key $ sk $ and the public key $ pk $, which are used in various encryption and evaluation operations.


In [3]:
# Keygen
pk, sk = rlwe_updated.keygen(n, q, poly_mod, std1)
pk, sk

((array([11799,  7390, 13643, 14142,   248,  8786,  1582, 15758],
        dtype=int64),
  array([ 6949, 10511, 11180, 11271,  4663,  1829, 15878, 14599],
        dtype=int64)),
 array([0, 0, 0, 1, 1, 0, 1, 1], dtype=int64))

# Encrypting the Binary Vectors

Once we have the binary vectors representing the characters in the message "Hello world," we proceed to encrypt each binary vector to obtain a set of ciphertexts. The encryption process involves converting each binary vector into an encrypted form using the public key $ pk $ and other encryption parameters.

$$
\begin{align*}
\text{ciphertexts} &: \text{List of encrypted binary vectors} \\
\text{binary\_vector} &: [\text{"01001000"}, \text{"01100101"}, \text{"01101100"}, \\
&\quad \text{"01101100"}, \text{"01101111"}, \text{"00100000"}, \text{"01110111"}, \\
&\quad \text{"01101111"}, \text{"01110010"}, \text{"01101100"}, \text{"01100100"}] \\
\text{pt} &: \text{Binary vector converted to a list of integers} \\
\text{ct} &: \text{Encrypted ciphertext obtained from the encryption function}
\end{align*}
$$

The resulting $ \text{ciphertexts} $ represent the encrypted form of the binary vectors corresponding to the characters in the message "Hello world." These ciphertexts preserve the privacy of the original message while allowing for further operations to be performed on the encrypted data.


In [5]:
# Encrypting "Hello world" as binary vectors
plaintext = "ahlem "
# Convert each character to 8-bit binary
binary_vector = [format(ord(char), '08b') for char in plaintext]
plaintext, binary_vector

('ahlem ',
 ['01100001', '01101000', '01101100', '01100101', '01101101', '00100000'])

In [6]:
# Encrypting the binary vectors
ciphertexts = []
for binary_value in binary_vector:
    pt = [int(bit) for bit in binary_value]
    ct = rlwe_updated.encrypt(pk, n, q, t, poly_mod, pt, std1)
    ciphertexts.append(ct)
ciphertexts

[(array([ 5568,  4807,  2860, 12498,  3523,  5588, 14255,  2773],
        dtype=int64),
  array([ 2148,  6229,  2195, 15290, 14520, 15391, 12891,  1120],
        dtype=int64)),
 (array([  358,  9949, 14087, 15951,  1503,  7894,    30, 13828],
        dtype=int64),
  array([  263, 14840, 12092, 14019,  6201, 16375,  8422, 10021],
        dtype=int64)),
 (array([ 7976,  9623, 14626,  9683,  8755, 13088,  3803, 15972],
        dtype=int64),
  array([11677,  5628, 12801,  3530, 12348,  9970,  7896, 15428],
        dtype=int64)),
 (array([11759,  7955,  6739,  9935, 10652,  3660, 13609,   987],
        dtype=int64),
  array([ 6330, 14394,  2200, 15342,  3911,  7642,  7346,  4714],
        dtype=int64)),
 (array([13977, 12926,  1400,  5767, 14834,  2651,  3431,  8256],
        dtype=int64),
  array([16190,  5655, 15005, 10398,   461,  9240,  2861, 12256],
        dtype=int64)),
 (array([10017,  7540,  6049,   785,  4648,  3493, 13800,  1020],
        dtype=int64),
  array([ 8540,  6730,  393

## Decrypting the Binary Vectors and Converting Back to Text

After encrypting the binary vectors representing the characters in the message "Hello world," we proceed to decrypt the ciphertexts and convert the decrypted binary vectors back into text. The decryption process involves using the secret key $ sk $ to reverse the encryption process and obtain the original binary vectors.

$$
\begin{align*}
\text{decrypted\_binary\_vectors} &: \text{List of decrypted binary vectors} \\
\text{ct} &: \text{Encrypted ciphertext to be decrypted} \\
\text{decrypted\_value} &: \text{Decrypted binary vector obtained from the decryption function}
\end{align*}
$$

By decrypting the ciphertexts, we recover the binary vectors that represent the characters in the original message. The resulting $ \text{decrypted\_binary\_vectors} $ are the binary representations of the characters "Hello world" before encryption.


In [7]:
# Decrypting the binary vectors and converting back to text
decrypted_binary_vectors = []
for ct in ciphertexts:
    decrypted_value = rlwe_updated.decrypt(sk, n, q, t, poly_mod, ct)
    decrypted_binary_vectors.append(decrypted_value)
decrypted_binary_vectors

[array([0, 1, 1, 0, 0, 0, 0, 1], dtype=int64),
 array([0, 1, 1, 0, 1, 0, 0, 0], dtype=int64),
 array([0, 1, 1, 0, 1, 1, 0, 0], dtype=int64),
 array([0, 1, 1, 0, 0, 1, 0, 1], dtype=int64),
 array([0, 1, 1, 0, 1, 1, 0, 1], dtype=int64),
 array([0, 0, 1, 0, 0, 0, 0, 0], dtype=int64)]

## Converting Decrypted Binary Vectors to Text

To obtain the final decrypted message, we convert the decrypted binary vectors back into their corresponding text representation. This involves reversing the process of converting characters to 8-bit binary vectors and mapping them back to their original characters.

$$
\begin{align*}
\text{decrypted\_text} &: \text{Decrypted message obtained from joining decrypted binary vectors}
\end{align*}
$$

The $ \text{decrypted\_text} $ represents the original message "Hello world," which has been successfully decrypted from its encrypted form.

In [8]:
decrypted_text = ''.join([chr(int(''.join(map(str, binary_vector)), 2)) for binary_vector in decrypted_binary_vectors])
decrypted_text

'ahlem '